In [1]:
import torch
import torchaudio
import torchaudio.sox_effects as sox
import numpy as np
from IPython.display import Audio


/opt/conda/envs/master_thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: libc10_cuda.so: cannot open shared object file: No such file or directory

In [ ]:
sox.init_sox_effects()

In [ ]:
sox.effect_names()

['allpass',
 'band',
 'bandpass',
 'bandreject',
 'bass',
 'bend',
 'biquad',
 'chorus',
 'channels',
 'compand',
 'contrast',
 'dcshift',
 'deemph',
 'delay',
 'dither',
 'divide',
 'downsample',
 'earwax',
 'echo',
 'echos',
 'equalizer',
 'fade',
 'fir',
 'firfit',
 'flanger',
 'gain',
 'highpass',
 'hilbert',
 'loudness',
 'lowpass',
 'mcompand',
 'norm',
 'oops',
 'overdrive',
 'pad',
 'phaser',
 'pitch',
 'rate',
 'remix',
 'repeat',
 'reverb',
 'reverse',
 'riaa',
 'silence',
 'sinc',
 'speed',
 'stat',
 'stats',
 'stretch',
 'swap',
 'synth',
 'tempo',
 'treble',
 'tremolo',
 'trim',
 'upsample',
 'vad',
 'vol']

In [ ]:
#Effect params
from torchaudio.utils.sox_utils import list_effects
list_effects()

{'allpass': 'frequency width[h|k|q|o]',
 'band': '[-n] center [width[h|k|q|o]]',
 'bandpass': '[-c] frequency width[h|k|q|o]',
 'bandreject': 'frequency width[h|k|q|o]',
 'bass': 'gain [frequency(100) [width[s|h|k|q|o]](0.5s)]',
 'bend': '[-f frame-rate(25)] [-o over-sample(16)] {start,cents,end}',
 'biquad': 'b0 b1 b2 a0 a1 a2',
 'chorus': 'gain-in gain-out delay decay speed depth [ -s | -t ]',
 'channels': 'number',
 'compand': "attack1,decay1{,attack2,decay2} [soft-knee-dB:]in-dB1[,out-dB1]{,in-dB2,out-dB2} [gain [initial-volume-dB [delay]]]\n\twhere {} means optional and repeatable and [] means optional.\n\tdB values are floating point or -inf'; times are in seconds.",
 'contrast': '[enhancement (75)]',
 'dcshift': 'shift [ limitergain ]\n\tThe peak limiter has a gain much less than 1.0 (ie 0.05 or 0.02) which\n\tis only used on peaks to prevent clipping. (default is no limiter)',
 'deemph': '',
 'delay': '{position}',
 'dither': '[-S|-s|-f filter] [-a] [-p precision]\n  (none)   U

In [ ]:
waveform, sample_rate = torchaudio.load('../datasets/GTZAN/gtzan_genre/genres/disco/disco.00017.wav')
waveform = waveform.mean(dim=0, keepdim=True)  # convert stereo to mono
Audio(waveform, rate=sample_rate)

In [ ]:
gain_min, gain_max = -12, 0
speed = np.random.uniform(0.5, 2.0)
pitch_min, pitch_max = -1200, +1200 #it ruins the lyrics!!!
reverberance, damping_factor, room_size = (np.random.randint(0, 100),) * 3

In [ ]:
effects = [
    ['gain', '-n', str(np.random.randint(gain_min, gain_max))],  # apply 10 db attenuation
    ['speed', f'{speed:.5f}'],  # duration is now 0.5 ~ 2.0 seconds.
    ['rate', f'{sample_rate}'],
    ['chorus', '0.8', '0.9', '55', '0.4', '0.25', '2', '-t'], #'chorus': 'gain-in gain-out delay decay speed depth [ -s | -t ]',
    ['overdrive', '30'], #'overdrive': '[gain [colour]]',
    ['pitch', str(np.random.randint(pitch_min, pitch_max))], #'pitch': 'semitones [octaves [cents]]',
    ['reverb', str(reverberance), str(damping_factor), str(room_size)], #'reverb': '[-w|--wet-only] [reverberance (50%) [HF-damping (50%) [room-scale (100%) [stereo-depth (100%) [pre-delay (0ms) [wet-gain (0dB)]]]]]]',
        'speed', '2'], #'speed': 'factor[c]',
    ['stretch', '1.5'], #'stretch': 'factor [window fade shift fading]\n       (expansion, frame in ms, lin/..., unit<1.0, unit<0.5)\n       (defaults: 1.0 20 lin ...)',
    ['tremolo', '10', '50'] #'tremolo': 'speed_Hz [depth_percent]',
    ]

In [ ]:
positive, sample_rate = sox.apply_effects_tensor(waveform, sample_rate, effects)
Audio(waveform, rate=sample_rate)